In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

Write a query to display for each order, the account ID, total amount of the order, and the level of the order - ‘Large’ or ’Small’ - depending on if the order is $3000 or more, or smaller than $3000.

In [6]:
pd.read_sql_query(sql='''
SELECT id, account_id, total_amt_usd,
CASE WHEN total_amt_usd < 3000 THEN 'less than 3000' ELSE '3000 or more' END AS ord_level
FROM orders
ORDER BY ord_level, total_amt_usd DESC;
''', con=conn)

,id,account_id,total_amt_usd,ord_level
0,4016,4251,232207.07,3000 or more
1,3892,4161,112875.18,3000 or more
2,3963,4211,107533.55,3000 or more
3,5791,2861,95005.82,3000 or more
4,3778,4101,93547.84,3000 or more
...,...,...,...,...
6907,6323,3551,0.00,less than 3000
6908,6375,3651,0.00,less than 3000
6909,6435,3801,0.00,less than 3000
6910,6523,3991,0.00,less than 3000


Solution:

SELECT account_id, total_amt_usd,
CASE WHEN total_amt_usd > 3000 THEN 'Large'
ELSE 'Small' END AS order_level
FROM orders;

Task 2:

Write a query to display the number of orders in each of three categories, based on the total number of items in each order. The three categories are: 'At Least 2000', 'Between 1000 and 2000' and 'Less than 1000'.

In [7]:
pd.read_sql_query(sql='''
SELECT CASE WHEN total < 1000 THEN 'Less than 1000'
WHEN total >= 1000 AND total < 2000 THEN 'Between 1000 and 2000'
ELSE 'At Least 2000' END AS ord_level, COUNT (*) AS ord_count
FROM orders
GROUP BY 1;
''', con=conn)

,ord_level,ord_count
0,At Least 2000,70
1,Between 1000 and 2000,511
2,Less than 1000,6331


Solution:

SELECT CASE WHEN total >= 2000 THEN 'At Least 2000'
   WHEN total >= 1000 AND total < 2000 THEN 'Between 1000 and 2000'
   ELSE 'Less than 1000' END AS order_category,
COUNT(*) AS order_count
FROM orders
GROUP BY 1;

Task 3:

We would like to understand 3 different levels of customers based on the amount associated with their purchases. The top level includes anyone with a Lifetime Value (total sales of all orders) greater than 200,000 usd. The second level is between 200,000 and 100,000 usd. The lowest level is anyone under 100,000 usd. Provide a table that includes the level associated with each account. You should provide the account name, the total sales of all orders for the customer, and the level. Order with the top spending customers listed first.

In [8]:
pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total_usd,
CASE WHEN SUM(total_amt_usd) > 200000 THEN 'Top level' WHEN SUM(total_amt_usd) > 100000 AND SUM(total_amt_usd) <= 200000 THEN 'Mid level' ELSE 'Low level' END AS cat
FROM accounts a
JOIN orders o
ON o.account_id = a.id
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,name,total_usd,cat
0,EOG Resources,382873.30,Top level
1,Mosaic,345618.59,Top level
2,IBM,326819.48,Top level
3,General Dynamics,300694.79,Top level
4,Republic Services,293861.14,Top level
...,...,...,...
345,Bed Bath & Beyond,1069.64,Low level
346,Deere,1036.57,Low level
347,Level 3 Communications,881.73,Low level
348,Delta Air Lines,859.64,Low level


Solution:

SELECT a.name, SUM(total_amt_usd) total_spent, 
     CASE WHEN SUM(total_amt_usd) > 200000 THEN 'top'
     WHEN  SUM(total_amt_usd) > 100000 THEN 'middle'
     ELSE 'low' END AS customer_level
FROM orders o
JOIN accounts a
ON o.account_id = a.id 
GROUP BY a.name
ORDER BY 2 DESC;

Task 4:

We would now like to perform a similar calculation to the first, but we want to obtain the total amount spent by customers only in 2016 and 2017. Keep the same levels as in the previous question. Order with the top spending customers listed first.

In [9]:
pd.read_sql_query(sql='''
SELECT a.name, SUM(total_amt_usd) total_usd,
CASE WHEN SUM(total_amt_usd) > 200000 THEN 'Top level'
WHEN SUM(total_amt_usd) > 100000 AND SUM(total_amt_usd) <= 200000 THEN 'Mid level'
ELSE 'Low level' END AS customer_level
FROM accounts a
JOIN orders o
ON o.account_id = a.id
WHERE o.occurred_at BETWEEN '2016-01-01' AND '2018-01-01'
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,name,total_usd,customer_level
0,Pacific Life,255319.18,Top level
1,Mosaic,172180.04,Mid level
2,CHS,163471.78,Mid level
3,Core-Mark Holding,148105.93,Mid level
4,Disney,129157.38,Mid level
...,...,...,...
317,Assurant,1235.81,Low level
318,Las Vegas Sands,1113.29,Low level
319,Deere,1036.57,Low level
320,Delta Air Lines,859.64,Low level


Solutions:

SELECT a.name, SUM(total_amt_usd) total_spent, 
     CASE WHEN SUM(total_amt_usd) > 200000 THEN 'top'
     WHEN  SUM(total_amt_usd) > 100000 THEN 'middle'
     ELSE 'low' END AS customer_level
FROM orders o
JOIN accounts a
ON o.account_id = a.id
WHERE occurred_at > '2015-12-31' 
GROUP BY 1
ORDER BY 2 DESC;

Task 5:

We would like to identify top performing sales reps, which are sales reps associated with more than 200 orders. Create a table with the sales rep name, the total number of orders, and a column with top or not depending on if they have more than 200 orders. Place the top sales people first in your final table.

In [10]:
pd.read_sql_query(sql='''
SELECT s.name, COUNT (*) total_orders,
CASE WHEN COUNT (*) > 200 THEN 'Top level' ELSE 'Other level' END AS sales_rep_level
FROM accounts a
JOIN orders o
ON o.account_id = a.id
JOIN sales_reps s
ON a.sales_rep_id = s.id
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,name,total_orders,sales_rep_level
0,Earlie Schleusner,335,Top level
1,Vernita Plump,299,Top level
2,Tia Amato,267,Top level
3,Georgianna Chisholm,256,Top level
4,Moon Torian,250,Top level
5,Nelle Meaux,241,Top level
6,Maren Musto,224,Top level
7,Dorotha Seawell,208,Top level
8,Charles Bidwell,205,Top level
9,Maryanna Fiorentino,204,Top level


Solution:

SELECT s.name, COUNT(*) num_ords,
     CASE WHEN COUNT(*) > 200 THEN 'top'
     ELSE 'not' END AS sales_rep_level
FROM orders o
JOIN accounts a
ON o.account_id = a.id 
JOIN sales_reps s
ON s.id = a.sales_rep_id
GROUP BY s.name
ORDER BY 2 DESC;

Task 6:

The previous didn't account for the middle, nor the dollar amount associated with the sales. Management decides they want to see these characteristics represented as well. We would like to identify top performing sales reps, which are sales reps associated with more than 200 orders or more than 750000 in total sales. The middle group has any rep with more than 150 orders or 500000 in sales. Create a table with the sales rep name, the total number of orders, total sales across all orders, and a column with top, middle, or low depending on this criteria. Place the top sales people based on dollar amount of sales first in your final table. You might see a few upset sales people by this criteria!

In [11]:
pd.read_sql_query(sql='''
SELECT s.name, COUNT (*) total_orders, SUM(o.total_amt_usd) total_usd,
CASE WHEN COUNT (*) > 200 OR SUM(o.total_amt_usd) > 750000 THEN 'Top level'
WHEN COUNT (*) > 150 AND COUNT (*) <= 200 OR SUM(o.total_amt_usd) > 500000 AND SUM(o.total_amt_usd) <= 750000 THEN 'Middle loevel'
ELSE 'Low level' END AS sales_rep_level
FROM accounts a
JOIN orders o
ON o.account_id = a.id
JOIN sales_reps s
ON a.sales_rep_id = s.id
GROUP BY 1
ORDER BY 2 DESC, 3 DESC;
''', con=conn)

,name,total_orders,total_usd,sales_rep_level
0,Earlie Schleusner,335,1098137.72,Top level
1,Vernita Plump,299,934212.93,Top level
2,Tia Amato,267,1010690.60,Top level
3,Georgianna Chisholm,256,886244.12,Top level
4,Moon Torian,250,650393.52,Top level
5,Nelle Meaux,241,749076.16,Top level
6,Maren Musto,224,702697.29,Top level
7,Dorotha Seawell,208,766935.04,Top level
8,Charles Bidwell,205,675637.19,Top level
9,Maryanna Fiorentino,204,655954.74,Top level


Solution:

SELECT s.name, COUNT(*), SUM(o.total_amt_usd) total_spent, 
     CASE WHEN COUNT(*) > 200 OR SUM(o.total_amt_usd) > 750000 THEN 'top'
     WHEN COUNT(*) > 150 OR SUM(o.total_amt_usd) > 500000 THEN 'middle'
     ELSE 'low' END AS sales_rep_level
FROM orders o
JOIN accounts a
ON o.account_id = a.id 
JOIN sales_reps s
ON s.id = a.sales_rep_id
GROUP BY s.name
ORDER BY 3 DESC;